In [4]:
from transformers import BertTokenizer
import numpy as np
from datasets import load_from_disk
from datasets import load_dataset

import random
import torch

from konlpy.tag import Mecab
mecab_tokenizer =Mecab().morphs

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [5]:
datasets = load_from_disk('/opt/ml/code/data/train_dataset')
training_dataset = datasets['train'][:]  

In [6]:
total_morph=[]
for sentence in training_dataset['context']:
    # 문장단위 mecab 적용
    token_mecab_save= mecab_tokenizer(sentence)
    total_morph.append(token_mecab_save)

In [7]:
kodataset=load_dataset("squad_kor_v1")
ko_training_dataset = datasets['train'][:]

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


In [8]:
for sentence in ko_training_dataset['context']:
    # 문장단위 mecab 적용
    token_mecab_save= mecab_tokenizer(sentence)
    total_morph.append(token_mecab_save)

In [9]:
with open('after_mecab.txt', 'w', encoding='utf-8') as f:
    for line in total_morph:
        f.write(' '.join(line)+'\n')

In [10]:
## 1) define special tokens
user_defined_symbols = ['[BOS]','[EOS]','[UNK0]','[UNK1]','[UNK2]','[UNK3]','[UNK4]','[UNK5]','[UNK6]','[UNK7]','[UNK8]','[UNK9]']
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
user_defined_symbols = user_defined_symbols + unused_list


In [11]:
import os
from tokenizers import BertWordPieceTokenizer, SentencePieceBPETokenizer, CharBPETokenizer, ByteLevelBPETokenizer

tokenizer = BertWordPieceTokenizer(strip_accents=False,  # Must be False if cased model
                                   lowercase=False)

corpus_file   = ['./after_mecab.txt']  # data path
vocab_size    = 32000
limit_alphabet= 6000
output_path   = 'hugging_%d'%(vocab_size)
min_frequency = 5


# Then train it!
tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,
               limit_alphabet=limit_alphabet,  
               show_progress=True)
print('train complete')

train complete


In [12]:
# save tokenizer
hf_model_path='tokenizer_model'
if not os.path.isdir(hf_model_path):
    os.mkdir(hf_model_path)
tokenizer.save_model(hf_model_path)  # vocab.txt 파일 한개가 만들어진다

['tokenizer_model/vocab.txt']

In [13]:
tokenizer_for_load = BertTokenizer.from_pretrained(hf_model_path,
                                                       strip_accents=False,  # Must be False if cased model
                                                       lowercase=False)  # 로드

In [14]:
# special token check
tokenizer_for_load.all_special_tokens # 추가하기 전 기본적인 special token

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [15]:
# tokenizer에 special token 추가
special_tokens_dict = {'additional_special_tokens': user_defined_symbols}
tokenizer_for_load.add_special_tokens(special_tokens_dict)

# check tokenizer vocab with special tokens
print('check special tokens : %s'%tokenizer_for_load.all_special_tokens[:20])

check special tokens : ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]']


In [16]:
# save tokenizer model with special tokens
tokenizer_for_load.save_pretrained(hf_model_path+'_special')

('tokenizer_model_special/tokenizer_config.json',
 'tokenizer_model_special/special_tokens_map.json',
 'tokenizer_model_special/vocab.txt',
 'tokenizer_model_special/added_tokens.json')

In [17]:
tokenizer = BertTokenizer.from_pretrained(hf_model_path+'_special')